IPython notebook created by BIOL3014 course staff. The material for the practical was created by Samir Lal, and extended by Michael McLeish as a BINF7000 major project in 2014.

Bug fixes: m.boden@uq.edu.au

Source: https://github.com/UQ-BIOL3014/Practical4

---

# BIOL3014/BINF7000 Practical 6

## Analysing gene expression using RNA-seq
---

* **Due:** 11AM 28/10/2015
* **Revision:** 1
* **Marks:** 
    * **BIOL3014** - 8 marks. 
    * (**BINF7000** - 12 marks. Not running in 2015.)

There are three parts, each with multiple problems to solve. BIOL3014 completes Parts 1 and 2 (labelled BIOL3014); BINF7000 completes Parts 1-3. (Not running in 2015.)

---


### Objectives 

In this practical:
* You learn about issues related to processing RNA-seq data
* You will learn about the limitations of certain statistical methods used in the analysis of this data
* You will learn about the different file formats involved in RNA-seq data analysis
* You learn practical approaches to analyse RNA-seq data
---


### Submission requirements

Please export this IPython notebook (with written answers & completed code) to `STUDENTNUMBER_P6.ipynb` notebook and submit it via the BlackBoard portal. See the screenshot below:

![alt text](export_workbook.png "Exporting your workbook")

----


### Resources

#### Python resources:
* The UQ Bioinformatics Python Guide (on Blackboard)
* The [Python 2 documentation]. For those unfamiliar with Python the [Official tutorial] is recommended
* The software carpentry [Novice python lessons]

[Python 2 documentation]: https://docs.python.org/2/
[Official tutorial]: https://docs.python.org/2/tutorial/index.html
[Novice python lessons]: http://swcarpentry.github.io/python-novice-inflammation/

####Relevant modules:
* `diff_exp.py` This submodule implements the differential expression methods
    of the sam_gmt project.
* `fileio.py` This submodule implements the functionality for interacting
    with file input and output.
* `plotting.py` This module implements the plotting and visualising used in
    the sam_gmt package.
* `perform_comparison.py`  This file executes the logic originally run globally at the
    bottom of the original sam.py file that the sam_gmt module is
    based off.
* `stats.py` This submodule implements the statistical and numerical methods
    of the sam_gmt project.
* `util.py` This submodule implements additional functions that
    are useful but not categorized within the other submodules.

#### Other:
* [IPython Notebook markdown cheatsheet]

[IPython Notebook markdown cheatsheet]: https://github.com/adam-p/markdown-here/wiki/Markdown-Here-Cheatsheet#links

---

## Exercises

For all example code to work, make sure you import all required modules. If Python reports an error for the code in the cell below, chances are that you don't have the code you need, or that Python does not find it. If the latter, make sure you set the Python search path *before* you start ipython notebook. 

There are references in the exercises to data files. These are generally kept in a sub-directory `data` of the current notebook directory.

The following `import` statements will assume that there is a further sub-directory `sam_gmt` with the necessary Python code.

In [1]:
from sam_gmt import diff_exp  # we are going to make reference to sequence alphabets
from sam_gmt import fileio    # for constructing sequences from genome reference data
from sam_gmt import plotting  # for processing genome-wide data
from sam_gmt import perform_comparison   # for motif data
from sam_gmt import stats     # statistics
from sam_gmt import util      # statistics

You will be using Numpy and matplotlib, so those standard libraries need to be imported too. We suggest that you make matplotlib plot its plots "in-line" so that visual results are available in the same notebook.

In [2]:
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

---
### Introduction
Within this practical you will be analysing RNA-seq data that has been generated by three scientists, Roger, Olivia and Grant. All three scientists have been working on a new antibiotic that works by attacking enyzmes produced in *Escherichia coli*. Previous differential expression experiments have shown that *E. coli* can regulate expression of genes when faced with foreign threats (Barbosa and Levy 2000, http://www.ncbi.nlm.nih.gov/pubmed/10852879).

---
### Part 1  RNA-seq read mapping (6 marks)

Roger, Olivia, and Grant have taken samples from the transcriptome at two different time points, one before and one after introduction of an antibiotic. They would like you to investigate if any genes are expressed at different rates between the two time points. They would also like you to report if there is a common cellular location between any differentially expressed genes, which might point to where the antibiotic is attacking the *E. coli* and how *E. coli* is able to defend itself.

<img src="E.coli.png">

*Figure 1: Diagram highlighting parts of *E. coli* (InfoSpace LLC 2014)*


#### Theory of RNA-seq
RNA-seq involves isolating RNA, randomly fragmenting the RNA, copying RNA into reads of the complementary DNA sequence, PCR amplification, and then the mapping of reads back onto a reference genome. Because we are interested in not just where they map to, but also the relative level of expression, we keep a record of how many reads map to each gene.

<img src="RNA-seq1.png">

*Figure 2: Different types and locations of RNA within cell (University of Kansas Medical Center 2014)*

<img src="RNA_seq_workflow.png">

*Figure 3: Basic overview of mRNA read mapping*

<span style="color:purple">Roger and his colleagues, Olivia and Grant, have been looking at RNA samples from *E.coli*. RNA samples were enriched for mRNA at two different time points (t1 and t10). An rRNA depletion method was used and you can be confident there is no rRNA appearing in the samples.

<span style="color:green">Olivia tells you that raw sequence reads of the mRNA were aligned to the *E.coli* reference genome using burrows wheeler alignment (see Figure 3). She tells you that the *E.coli* reference genome is well documented.

<span style="color:blue">Grant mentions that he is aware that RNA-seq can be used to discover exon/intron boundaries. He isn’t sure if this is relevant to your experiment with *E.coli* and suggests you might want to look into it.

<span style="color:red">**Prac Question One: Consider the experimental process for performing an RNA-seq analysis as outlined above. Identify two broad areas where errors in data could occur and what effect this might have on our results.**


<span style="color:blue">Grant: I didn’t really understand the Burrows-Wheeler transform, so I had a read of the papers online and I think I have a better handle on it now.

<img src="BW_Alignment.png">

*Figure 4: Diagram showing rotated and transformed string using the Burrows-Wheeler transform (Langmead 2013)*

Read the Wikipedia entry on Burrows-Wheeler transforms (http://en.wikipedia.org/wiki/Burrows%E2%80%93Wheeler_transform) 
or ‘Introduction to the Burrows-Wheeler Transform and FM Index’ (Langmead 2013, http://www.cs.jhu.edu/~langmea/resources/bwt_fm.pdf) to get a better understanding of how it is performed.

The sequence reads aligned to the *E. coli* genome are stored as sequence alignment map (SAM) files; these files can also be stored as more memory efficient binary SAM files (BAM). For more information on the SAM format visit http://samtools.github.io/hts-specs/SAMv1.pdf.


#### Biological data type I: SAM File
<span style="color:green">Olivia: Load my first SAM file which contains reads mapped at time point 1. I also created replicates of the data at each time point. We expect these replicates to be similar to their corresponding time point, so it is important to consider them as it helps us quantify the amount of experimental noise that is present.


<span style="color:green">Olivia: Let’s get a feel for the reads that are contained in this SAM file. The functions mapped_bases(), and base_percentages() will give you a good starting point to find out some general information.

<span style="color:red">**Prac Question Two: How many reads (total and aligned) are in the file? What percentage of reads has been aligned? What are the nucleotide frequencies of the total reads?**

In [3]:
t1_olivia = fileio.read_sam("data/t1.sam")
t1_olivia_replicate = fileio.read_sam ("data/t1_2.sam")


<span style="color:green">Olivia: SAM files contain many columns of data. One very important column is the 5th column that contains the mapping quality (MAPQ) (see Figure 5).

```
MAPQ = 10 × log(10) x Probability{Mapping position is wrong}
```

<img src="SAM_file_example.png">

*Figure 5: Columns of a SAM file with MAPQ column highlighted*



<span style="color:green">Olivia: I’d like to use these MAPQ values to give an idea of the probability of an incorrect mapping. Can you write Python code that extracts these MAPQ values? I’d like to have the range of MAPQ values and their probabilities. The method `inv_logit()` in `stats.py` might be useful here.

<span style="color:red">**Prac Question Three: Print the range of MAPQ values in Olivia’s time point 1 data with their corresponding probabilities.**

<span style="color:green">Olivia: Thanks! But I still need to group the reads that I have into three distinct subgroups to determine the composition of probabilities I have.

<span style="color:red">**Prac Question Four: The function `subgroups()` in the `util` module has been supplied for you which will sort reads into groups based on the probability that the read is incorrect. Go through the code below and comment each line so it is clear what the function is doing at each step.**

In [12]:
def sub_groups(mapped_reads):
    """ Return a tuple of three "groups",
        seperated by p<1e-3, 1e-3<=p<1e-2 and 1e-2<=p<1
    """
    groups = [], [], []

    for read in mapped_reads:
        read_val = int(read[4])
        if read_val > 29:
            group_index = 0
        elif 17 < read_val <= 29:
            group_index = 1
        elif read_val <= 17:
            group_index = 2

        groups[group_index].append(read)

    return groups

<span style="color:green">Olivia: Ah that’s a lot clearer! Because I’m interested in the relative percentages of the reads, creating a pie chart would be a quick and easy way to visually display this data. And to check that this data remains fairly consistent, can you also plot the same pie chart for the replicate I created at time point 1.

<span style="color:red">**Prac Question Five: Using the function `aligned_reads()` in `plotting.py` create pie charts for time point 1 and the time point 1 replicate. Comment on the similarities and differences between these two charts.**

<span style="color:green">Olivia: Lets report on the different lengths that occur in the `t1.sam` file in the `data` directory. Provide the mean, minimum, and maximum length of sequence reads that occur for each subgroup you created earlier.

<span style="color:red">**Prac Question Six: As well as the mean, minimum, and maximum length of the reads, provide Olivia with a histogram showing the distribution of the lengths in each of the three subgroups.**

Once we have an idea of the relative abundance and length of the reads, we want to map them to the *E.coli* genome and get an idea of the raw counts of SAM reads in each mapped position in the *E.coli* genome (see Figure 7).

<img src="SAM_reads_mapped.png">
*Figure 7: SAM reads mapped to *E.coli* genome*

Raw counts have been provided for you in the file `olivia_raw_counts.txt`. Using the function `read_rawcounts` in `fileio`, you can map the raw counts for Olivia’s data at time point 1, the time point 1 replicate, time point 10, and the time point 10 replicate.

### Part 2  RNA-seq normalisation (4 marks)
<span style="color:purple">Roger: These raw counts can’t be compared directly yet, because they could be suffering from the variations that I mentioned earlier. We need to normalise the data first. Normalisation of data is the process of transforming data that has come from two or more different sources by scaling them to a common factor. In this way we can then directly compare them. Sometimes when numbers would be very small or very large we also scale them into a range of numbers that are easier for human minds to comprehend, for example a pH scale.

<span style="color:purple">Roger: Normalising our data by using RPKM allows for differences in gene length and differences in sequencing depth to minimised, so we can compare across samples.

<img src="Diagram_of_different read_lengths.png">
*Figure 8: Diagram showing relationship between length, depth and count (Tarazona 2011)*

An issue to be aware of with sequencing depth is that we quickly reach a high number of genes discovered and the detection number plateaus after a certain point. This has the implication that past a certain point increasing our sequencing depth will have a significant cost for not much benefit.

<img src="compare_norms.png">
*Figure 9: Number of Open Reading Frames detected vs Number of uniquely mapped tags (million) (Wang, Gerstein & Snyder 2009)*

<span style="color:red">**Prac Question Seven: A commonly used normalisation technique in RNA-seq is Reads Per Kilobase per Million (RPKM). Provide the formula for this normalisation function. With the formula you found, show the calculation for the RPKM score of a gene that has 300 reads mapped to it and is 250 base pairs long. Assume that the total mapped reads is 100 million. To ensure you have the formula correct, your answer should give an RPKM of 12.**

<span style="color:green">Olivia: You can use `olivia_raw_counts.txt` in the `data` directory and the method `get_rpkm` in `util.py` to calculate the RPKM value for my data. Use a num map of (118898, 121634, 136286, 135102) to get the correct number of mapped reads:

```
olivia_raw = fileio.read_rawcounts("data/olivia_raw_counts.txt")
num_map = (118898, 121634, 136286, 135102)
rpkm = util.get_rpkm(olivia_raw, num_map)
```

<span style="color:green">How can we tell if our normalisation process has had an effect on our data? We will need to graph the data pre and post normalisation in a way that allows seeing if the variation in the data has been reduced. One way we can check this is through the coefficient of variation / standard error.

```
Coefficient of variation (transcript) = (standard deviation (transcript)) / (mean (transcript))
```

<img src="cov_sterr.png">
*Figure 10: Coefficient of variation (Brighton Webs 2012)*

As we can see from Figure 10, even with different means and standard deviations we can use the coefficient of variation to see the degree of variation between two datasets.

<span style="color:red">**Prac Question Eight: Using the methods `get_cv` in `util.py` and `box_plots` and `average_cv` in `plotting.py`, provide boxplots of the data before and after RPKM normalisation showing the distribution of variation at time point 1. Plot the average coefficient of variation at time point 1 before and after RPKM normalisation. Comment on the results and differences.**


<span style="color:green">Olivia: We need to have a way of deciding if variation that exists between time points 1 and 10 is related to biological reasons or simply errors in our ability to reproduce data. Plotting my time point 1 data against my time point 1 replicate should give an idea of how reproducible the experimental procedure was (see Figure 11).

<img src="replicates.png">
*Figure 11: Reproducibility of RNA-seq experiments expressed via technical replicates mapped against each other (BGI 2014)*




<span style="color:blue">Grant: I remembered to take replicates. Can you map my time point 1 and time point 1 replicate data?

<span style="color:red">**Prac Question Nine: Plot Olivia’s replicates at time point 1 and provide the graph in your report `plotting.rpkm_repr(rpkm, ‘t1’)`. Raw counts of Grant’s reads are provided in `grant_raw_counts.txt`. Perform the same steps you did for Olivia’s raw counts, and then plot the replicates against each other (producing a version of Figure 11). What could explain the shape of this graph?**

Another way to identify reproducibility is to plot the difference between two log transformed intensities (M) and the average log intensity (A). We can easily see any points that are not showing differential expression because they will be situated at 0.

Because, when t1 expression is the same as t10

log (t1 / t10) becomes log (1 / 1)

log (1) = 0



RPKM normalisation can create a bias if there are a few genes showing high levels of expression in two states - these genes dominate the RPKM equation but there might not be biological reason to assume that they are both constants (Cai et al, 2012).
Wagner and Lynch (2012) explore this by highlighting that that the total read number is something that has meaning within the individual runs of sequencing, but that it is not a statistic that has larger biological relevance, i.e. we cannot infer actual RNA levels from it.

<span style="color:green">Olivia: Now that we have normalised our counts we can begin to compare them. We are interested in genes that are either up-regulated or down-regulated between the two time points (1.5 or -1.5 fold change).

<span style="color:red">**Prac Question Ten: Plot Olivia’s time point 1 and time point 10 data using ma() which you will find within plotting.py.  Identify some differentially expressed genes and some genes that are not differentially expressed.**

---
### Part 3 Gene enrichment (2 marks)

<span style="color:green">Olivia: So here is our list of potential differentially expressed transcripts, but we still need to minimise the risk that we have selected a transcript by chance.

Let’s say we have a p-value of 0.01, meaning there is a 1% chance that a result is a false positive. The problem that arises when doing multiple tests is that this chance grows with the number of tests. Using this scale, in 2000 tests we would expect to get 20 false positives, an issue that is known as the multiple comparisons problem (http://en.wikipedia.org/wiki/Multiple_comparisons_problem).

What we need to do is calculate a q-value to help minimise the false discovery rate. A p-value of 0.01 implies there is a 1% chance across all results, but a q-value of 0.01 implies there is a 1% chance across only the statistically significant results. This will result in a smaller amount of samples expected to be occurring due to random chance.

<img src="false_discovery_rate.png">
*Figure 12 Limiting effect that q-value has on sample space for false discovery rate (TotalLab 2013)*

We need to check to see if the difference between the means of time point 1 and time point 10 are significant enough to not have occurred purely to chance.

<span style="color:green">Olivia: Controlling the false discovery rate by creating a q-value which is the minimum false discovery rate we will accept allows for more statistically rigorous analysis.

<span style="color:red">**Prac Question Eleven: Using the function welch_t_test(), which calls the function correct_pvalues(), provide a list of differentially expressed transcripts that have statistical significance.**


<span style="color:green">Olivia: Now that we have our list of transcripts we can answer the fundamental question about whether these transcripts have a common biological function or cellular location.

One way of looking at the function of gene products is to search databases for common functional groups present within the set of products. 

We can use a site such as the Database for Annotation, Visualisation and Integrated Discovery (DAVID) to work out the functionality of our transcripts. At the moment we have a list of transcripts, so we will need to transform this list into a list of identifiers that can search DAVID.

The UniProt Knowledgebase is a database that provides functional information about proteins. We can upload our list of transcripts and convert them into a list of gene names. Go to http://www.uniprot.org/uploadlists. Paste the transcripts you identified as being statistically significant into the identifiers window, and under ‘Select options’ you will need to select From: Ensembl Genomes and To: UniProtKB. 

In the column Gene names there are a series of entries consisting of gene names, where available. These are the bolded items in this column and we want a list only comprising the bold terms. Have a look at some of the columns that don’t contain a bold term. 

<img src="genes_random.png">

*Figure 13 Column view of UniProt (UniProt 2014)*

Click on the Columns button and under Names & Taxonomy investigate which option will just give the bold gene names.

<img src="taxonomy.png">

*Figure 14 Names & Taxonomy view of UniProt (UniProt 2014)*

Either download a copy of the gene names from this column, or simply copy the list of gene names to your computer’s clipboard.
Now we’ll analyse the function of the genes we’re identified. Go to the site DAVID (Database for Annotation, Visualization and Integrated Discovery) at http://david.abcc.ncifcrf.gov/ 

1.	Along the left hand side access the Gene Functional Classification Tool
2.	Paste the list of gene names into the field provided
3.	Select OFFICIAL_GENE_SYMBOL as the Identifier
4.	Select Gene List as the list type
5.	Submit the list

We can see that multiple organisms give hits for the gene names we’re searching for. Select *Escherichia coli* and then observe the genes that have been found to be similar through an enrichment analysis.

Click on the black and green heat map symbol indicated by the arrow in Figure 15.

<img src="Gene_functions.png">
*Figure 15: Gene Functional Classification Result*

Using Figure 16 as guide to the layout of the heat map, analyse the results you have and answer the final Prac Question.

<img src="heatmap_explained.png">

*Figure 16 2D Heat Map View from DAVID (DAVID 2014)*

<span style="color:red">**Prac Question Twelve: Provide the heat map you generated. Comment on the types of genes that were enriched and report on possible cellular location of the differentially expressed genes.**

---
####References
Barbosa, TM & Levy, SB, Differential expression of over 60 chromosomal genes in Escherichia coli by constitutive expression of MarA, Journal of Bacteriology, vol. 182, no.12, pp. 3467 – 3474.
BGI 2014, RNA-seq quantification, viewed 10 October 2014, <http://www.innolifetech.com/pdf/RNA-Seq%20%28Quantification%29.pdf>.

Brighton Webs 2012, Coefficient of Variation, viewed 10 October 2014, <http://www.brighton-webs.co.uk/statistics/coeff_of_variation.aspx>.

DAVID 2014, DAVID Functional Annotation Bioinformatics Microarray Analysis, viewed 10 October 2014, < http://david.abcc.ncifcrf.gov/>. 

InfoSpace LLC 2014, How Evolution Works, viewed 13 October 2014, <http://science.howstuffworks.com/life/evolution/evolution4.htm>.

Langmead, B 2103, Introduction to the Burrows-Wheeler Transform and FM Index, Department of Computer Science, JHU, viewed 16 October 2014, <http://www.cs.jhu.edu/~langmea/resources/bwt_fm.pdf>.

Tarazona, S 2011, Differential expression in RNA-Seq, PowerPoint Slides, Centro de Investigacion Principe Felipe, Spain.
TotalLab 2013, What are p-values? What are q-values, and why are they important?, viewed 9 October 2014, <http://www.totallab.com/products/samespots/support/faq/pq-values.aspx>. 

UniProt 2013, UniProt, viewed 9 October 2014 <http://www.uniprot.org/>. 

University of Kansas Medical Center 2014,The nucleus tutorial – nuclear pores and transport, viewed 12 October 2014, <http://classes.kumc.edu/som/cellbiology/organelles/nucleus/tut2.html>.

Wang, Z, Gerstein, M & Snyder, M 2009, RNA-Seq: a revolutionary tool for transcriptomics, Nature Reviews Genetics, vol. 10, no. 1, pp. 57-53.


